# TP Académico agente experto de Eventos - IA

## Importación de librerías
Se cargan los módulos principales de LangChain, OpenAI, manejo de bases de datos (SQLAlchemy, pandas) y herramientas auxiliares.

In [1]:
# Si estás en Colab y te faltan paquetes, descomenta y ejecuta:
!pip -q install "langchain>=0.2" langchain-openai langchain-community ipywidgets \
                 sqlalchemy psycopg2-binary pandas tabulate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 383.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 810.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
# --- imports base ---
import re
from io import StringIO
import pandas as pd
from typing import Literal

# --- LangChain principal ---
from langchain.agents import create_react_agent, AgentExecutor
from langchain.prompts import PromptTemplate
from langchain.tools import tool

# --- Conectores de modelos (OpenAI) ---
from langchain_openai import ChatOpenAI

# --- Memoria de conversación
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


## Configuración de credenciales y backend
Se configura la **clave de autenticación de OpenAI** como variable de entorno (`OPENAI_API_KEY`), la cual permite el acceso seguro al modelo de lenguaje utilizado por el agente.  

También se define el backend a emplear (`openai`), indicando que el sistema trabajará exclusivamente con la API oficial de OpenAI como proveedor del modelo GPT.  

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

# Verificación de seguridad
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("Por favor, configura la OPENAI_API_KEY en el archivo .env")

# Configuración
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
BACKEND: Literal["openai"] = "openai"


## Función de creación del modelo
Esta función encapsula la creación del modelo. Luego, la variable con el modelo se utiliza en el resto del código.

En esta, se define la creación del **modelo de lenguaje (LLM)** utilizado por el agente.  

La función `make_llm()` se encarga de instanciar el modelo, configurando su **temperatura** (nivel de creatividad en las respuestas) y el backend correspondiente.


In [4]:
def make_llm(
    backend: Literal["openai"] = "openai",
    temperature: float = 0.2,
    openai_model: str = "gpt-4o-mini",
):
    if backend == "openai":
        return ChatOpenAI(model=openai_model, temperature=temperature)
    raise ValueError("Backend no soportado")


## Configuración de la base de datos
En esta sección se configura la conexión con la base de datos (Supabase), un entorno PostgreSQL administrado en la nube.  
Se definen las credenciales de acceso y se construye la cadena de conexión (`DB_URL`).

También se implementa la función `safe_select()`, que garantiza que el sistema solo pueda ejecutar **sentencias de lectura (SELECT o CTE)**, impidiendo cualquier intento de modificación de datos.  


In [ ]:
from dotenv import load_dotenv
# === Config DB ===
from urllib.parse import quote_plus
from sqlalchemy import create_engine, text as sql_text

# Cargar las variables del archivo .env
load_dotenv()

# Leer las credenciales de forma segura
SUPABASE_USER = os.getenv("SUPABASE_USER")
SUPABASE_PASS = os.getenv("SUPABASE_PASS")
SUPABASE_HOST = os.getenv("SUPABASE_HOST")
SUPABASE_PORT = os.getenv("SUPABASE_PORT")
SUPABASE_DB   = os.getenv("SUPABASE_DB")
if not SUPABASE_PASS:
    raise ValueError("¡Error! Falta configurar las credenciales en el archivo .env")

PASS_ENC = quote_plus(SUPABASE_PASS)
DB_URL = f"postgresql+psycopg2://{SUPABASE_USER}:{PASS_ENC}@{SUPABASE_HOST}:{SUPABASE_PORT}/{SUPABASE_DB}?sslmode=require"

os.environ["DB_URL"] = DB_URL

engine = create_engine(DB_URL, pool_pre_ping=True)

def safe_select(sql: str) -> pd.DataFrame:
    """
    SOLO permite SELECT/CTE (WITH). Usa comillas dobles en NombresConMayúsculas.
    """
    sql_clean = sql.strip().lower().lstrip("`").lstrip("--").strip()
    if not (sql_clean.startswith("select") or sql_clean.startswith("with")):
        raise ValueError(f"Solo se permiten consultas SELECT/CTE (lectura).\nQuery rechazada: {sql}")

    # 🔧 Limpieza de comillas o punto y coma al final
    sql = sql.strip().rstrip(';').rstrip('"').rstrip("'")

    print("[DEBUG] Ejecutando query segura:\n", sql)
    with engine.connect() as conn:
        return pd.read_sql(sql_text(sql), conn)

def run_sql_df(sql: str) -> pd.DataFrame:
    """
    Igual que run_sql_tool(), pero devuelve un DataFrame (no texto)
    para usar en funciones internas como gráficos o cálculos.
    """
    try:
        return safe_select(sql)
    except Exception as e:
        raise RuntimeError(f"[SQL ERROR] {e}")

In [8]:
safe_select("SELECT 1 AS ok")


[DEBUG] Ejecutando query segura:
 SELECT 1 AS ok


,ok
0,1


## Configuraciones de las Tools + funciones necesarias
Se configuran las tools que tendra el agente y otras funciones para agregar funcionalidad.

Cada tool representa una acción específica que el agente puede ejecutar según la necesidad detectada durante su razonamiento ReAct.

Finalmente, todas las herramientas se agrupan en la lista global `TOOLS`, que será incorporada al agente para que las utilice de forma dinámica según el contexto de la consulta. Este diseño modular permite extender fácilmente las capacidades del agente con nuevas funciones en el futuro.

In [ ]:
# 1) --- SELECT (solo lectura) ---
@tool
def sql_select(query: str) -> str:
    """Ejecuta consultas SQL **solo SELECT** sobre la base Supabase.
    Devuelve una tabla (markdown) y un CSV completo como texto."""
    try:
        df = safe_select(query)
    except Exception as e:
        return f"[SQL ERROR] {e}"
    if df.empty:
        return "Sin resultados."
    md = df.head(30).to_markdown(index=False)
    csv_ = df.to_csv(index=False)
    return f"Vista previa (30 filas):\n{md}\n\nCSV completo:\n{csv_}"

# 2) --- Lista de eventos ---
@tool
def lista_eventos(_: str = "") -> str:
    """Lista id, nombre, fecha y capacidad de los eventos (ordenados por fecha desc)."""
    q = 'SELECT "idEvento","nombre","fechaHoraEvento","capacidadMax" FROM "Evento" ORDER BY "fechaHoraEvento" DESC;'
    return sql_select.run(q)


# 3) --- Disponibilidad de un evento ---
@tool
def disponibilidad_evento(filtro: str) -> str:
    """
    Devuelve disponibilidad por evento (capacidad, vendidos, cupos).
    Acepta un id numérico o parte del nombre (con o sin comillas).
    Excluye tickets 'reembolsado'.
    """
    try:
        raw = (filtro or "").strip()
        # sacamos comillas si vienen "'Oasis'" o '"Oasis"'
        raw = raw.strip("'").strip('"')

        by_id = re.fullmatch(r"\d+", raw) is not None
        if by_id:
            where = 'e."idEvento" = :id'
            params = {"id": int(raw)}
        else:
            where = 'e."nombre" ILIKE :name'
            params = {"name": f"%{raw}%"}

        sql = f"""
        WITH base AS (
            SELECT e."idEvento", e."nombre", e."capacidadMax"
            FROM "Evento" e
            WHERE {where}
        ),
        vendidos AS (
            SELECT e."idEvento", COUNT(*)::int AS tickets_emitidos
            FROM "Ticket" t
            JOIN "TipoTicket" tt ON t."idTipoTicket" = tt."idTipoTicket"
            JOIN "Evento" e ON e."idEvento" = tt."idEvento"
            WHERE {where}
              AND t."estado" <> 'reembolsado'
            GROUP BY e."idEvento"
        )
        SELECT b."idEvento",
               b."nombre",
               b."capacidadMax",
               COALESCE(v.tickets_emitidos, 0) AS tickets_emitidos,
               GREATEST(b."capacidadMax" - COALESCE(v.tickets_emitidos, 0), 0) AS cupos_disponibles
        FROM base b
        LEFT JOIN vendidos v ON v."idEvento" = b."idEvento"
        ORDER BY b."idEvento";
        """

        with engine.connect() as conn:
            df = pd.read_sql(sql_text(sql), conn, params=params)

        if df.empty:
            return "No se encontró ningún evento con ese filtro."
        return df.to_markdown(index=False)

    except Exception as e:
        return f"[SQL ERROR] {e}"


# 4) --- Política vigente ---
@tool
def politica_vigente(_: str = "") -> str:
    """Devuelve la política de reembolso más reciente."""
    try:
        q = 'SELECT "fechaVigencia","tiempoReembolso" FROM "Politica" ORDER BY "fechaVigencia" DESC LIMIT 1;'
        df = safe_select(q)
        if df.empty:
            return "No hay políticas cargadas."
        r = df.iloc[0]
        return f"Política vigente desde {r['fechaVigencia']:%Y-%m-%d}: reembolso dentro de {int(r['tiempoReembolso'])} días."
    except Exception as e:
        return f"[SQL ERROR] {e}"

# 5) --- Organizador con más eventos ---
@tool
def organizador_mas_eventos(_: str="") -> str:
  """  Devuelve un ranking de los organizadores (tabla Organizacion)
    con mayor cantidad de eventos registrados en la base de datos.
    Usa la tabla Evento y la relación Evento.idOrganizacion -> Organizacion.idOrganizacion."""
  try:
    q= 'SELECT o."nombre" AS organizador, COUNT(e."idEvento") AS cantidad_eventos FROM "Evento" e LEFT JOIN "Organizacion" o ON e."idOrganizacion" = o."idOrganizacion" GROUP BY o."nombre" ORDER BY cantidad_eventos DESC;'
    df =safe_select(q)
    if df.empty:
      return "No hay eventos registrados para ninguna organizacion"
    return sql_select.run(q)
  except Exception as e:
      return f"[SQL ERROR] {e}"


# 6) --- Recomendación por presupuesto ---
@tool
def recomendar_por_presupuesto(monto_str: str) -> str:
    """Recibe un monto (ej. '15000' o '15.000').
    Lista eventos y tipos de ticket cuyo precio <= monto."""
    try:
        monto = float(re.sub(r"[^\d.,]", "", monto_str).replace(".", "").replace(",", "."))
    except:
        return "Pasá un monto en pesos, ej: 15000"
    q = f"""
    SELECT e."idEvento", e."nombre" AS evento, e."fechaHoraEvento",
           tt."idTipoTicket", tt."tipo", tt."precio"
    FROM "TipoTicket" tt
    JOIN "Evento" e ON e."idEvento" = tt."idEvento"
    WHERE tt."precio" <= {monto}
    ORDER BY e."fechaHoraEvento" ASC, tt."precio" DESC;
    """
    return sql_select.run(q)


# 7) --- Resumen ejecutivo a partir de CSV ---
@tool
def resumen_ejecutivo(csv_text: str) -> str:
    """Dado un CSV (salida de sql_select), redacta un resumen ejecutivo de hallazgos."""
    try:
        df = pd.read_csv(StringIO(csv_text))
    except Exception as e:
        return f"[CSV ERROR] {e}"
    info = [f"Filas={len(df)}; Columnas={list(df.columns)}"]
    for c in ["precio", "capacidadMax"]:
        if c in df.columns:
            info.append(f"{c}: mean={df[c].mean():.2f}  min={df[c].min():.2f}  max={df[c].max():.2f}")
    if "estado" in df.columns:
        info.append("Tickets por estado:\n" + df["estado"].value_counts().to_string())
    llm = make_llm()
    return llm.invoke("Redacta un resumen ejecutivo claro y conciso a partir de:\n" + "\n".join(info)).content



# 8) --- Grafica Tickets Vendidos  ---
@tool
def grafico_tickets_vendidos(_: str = "") -> str:
    """
    Genera un gráfico de barras con la cantidad de tickets vendidos por evento.
    Usa internamente la consulta SQL correspondiente (no requiere entrada).
    Guarda el gráfico en /tmp/grafico_tickets_evento.png y devuelve un resumen de datos.
    """
    try:
        import matplotlib.pyplot as plt
        import os

        print("[DEBUG] Ejecutando query para gráfico de tickets vendidos...")
        q = """
        SELECT e."nombre", COUNT(t."nroTicket") AS tickets_vendidos
        FROM "Evento" e
        LEFT JOIN "TipoTicket" tt ON e."idEvento" = tt."idEvento"
        LEFT JOIN "Ticket" t ON tt."idTipoTicket" = t."idTipoTicket"
        GROUP BY e."nombre"
        ORDER BY tickets_vendidos DESC;
        """

        # usamos run_sql_df si ya la tenés definida, o safe_select si preferís consistencia
        df = run_sql_df(q)

        print("[DEBUG] Query completada. Filas:", len(df))
        if df.empty:
            return "No hay datos para graficar."

        # --- generar gráfico ---
        plt.figure(figsize=(10, 6))
        plt.bar(df["nombre"], df["tickets_vendidos"])
        plt.xticks(rotation=45, ha='right')
        plt.title("Tickets vendidos por evento")
        plt.xlabel("Evento")
        plt.ylabel("Cantidad de Tickets")
        plt.tight_layout()

        # guardar imagen en carpeta temporal
        path = os.path.join("/tmp", "grafico_tickets_evento.png")
        plt.savefig(path)
        plt.close()

        # devolver datos + path
        datos_str = df.to_string(index=False)
        return (
            f"Datos de tickets vendidos por evento:\n{datos_str}\n\n"
            f"Gráfico guardado en: {path}"
        )

    except Exception as e:
        import traceback
        return f"[ERROR] No se pudo generar el gráfico:\n{traceback.format_exc()}"

# 9) --- Lista los organizador que tienen mayor cantidad de tickets vendidos ---
@tool
def organizadores_por_recaudacion(_: str = "") -> str:
    """Muestra los organizadores con mayor cantidad de tickets vendidos y total recaudado."""
    try:
        q = """
        SELECT
          o."nombre" AS organizador,
          COUNT(t."nroTicket") AS tickets_vendidos,
          SUM(tt."precio") AS total_recaudado
        FROM "Organizacion" o
        JOIN "Evento" e ON o."idOrganizacion" = e."idOrganizacion"
        JOIN "TipoTicket" tt ON e."idEvento" = tt."idEvento"
        JOIN "Ticket" t ON tt."idTipoTicket" = t."idTipoTicket"
        GROUP BY o."nombre"
        ORDER BY total_recaudado DESC;
        """
        return sql_select.run(q)
    except Exception as e:
        return f"[SQL ERROR] {e}"

#10) --- Lista los eventos pertenecientes al nombre de una Categoria específica
@tool
def categoria_evento(nombre_cat:str)-> str:
  """Dado el nombre (o parte del nombre) de una categoria ingresado por el usuario, listar todos los eventos asociados a esa categoria mostrando su nombre y descripcion."""
  try:
    categoria = nombre_cat.replace("'", "''").strip()
    q= f"""
       SELECT
            c."nombreCategoria" AS categoria,
            e."nombre" AS evento,
            e."descripcion"
        FROM "Evento" e
        INNER JOIN "Categoria" c ON e."idCategoria" = c."idCategoria"
        WHERE c."nombreCategoria" ILIKE '%{categoria}%'
        ORDER BY e."fechaHoraEvento" ASC;

    """
    df= safe_select(q)
    if df.empty:
      return f"No hay eventos registrados para la categoria '{nombre_cat}'."
    return df.to_markdown(index=False)
  except Exception as e:
    return f"[SQL ERROR] {e}"

# --- Lista global para registrar todas las herramientas ---
TOOLS = [
    sql_select,
    lista_eventos,
    disponibilidad_evento,
    politica_vigente,
    recomendar_por_presupuesto,
    resumen_ejecutivo,
    grafico_tickets_vendidos,
    organizador_mas_eventos,
    organizadores_por_recaudacion,
    categoria_evento,
]

## Definición del Promt
Se define el prompt maestro que guía el comportamiento del agente bajo el patrón **ReAct (Reason + Act)**.  

El template establece:

- el **idioma y tono** (español rioplatense, profesional)

- el **formato de interacción** (Question → Thought → Action → Observation → Final Answer)

- las **reglas de seguridad** (solo lectura en la base, no inventar observaciones)

- el **enrutamiento** hacia las herramientas disponibles según el tipo de consulta.

Este prompt se materializa en un `PromptTemplate` de LangChain (`prompt = PromptTemplate.from_template(template)`), que el agente utilizará en cada turno para mantener consistencia, trazabilidad y control del flujo de acciones.



In [10]:
template = """You are an intelligent data analyst agent working for a ticketing and events company.
Your goal is to always respond in Rioplatense Spanish, using a professional and clear tone,
and to use the available tools whenever necessary.

Conversación hasta ahora:
{chat_history}

Nota: si chat_history está vacío, ignora esta sección.

Seguí EXACTAMENTE este formato:

Question: la pregunta del usuario
Thought: explicación breve de tu plan de acción
Action: el nombre de UNA herramienta, exactamente de [{tool_names}]
Action Input: la entrada para esa herramienta
Observation: el resultado de la herramienta
... (repetí Thought/Action/Observation si necesitás ejecutar más de una herramienta)
Final Answer: respuesta final breve y clara en español

Reglas generales importantes:
- Si elegís una Action, NO escribas 'Final Answer' en el mismo turno.
- Nunca inventes resultados: solo escribí "Observation" después de ejecutar una tool.
- Si no necesitás herramientas, podés ir directo a "Final Answer".
- Si una tool devuelve una tabla grande o CSV, mostrala como **tabla corta** y agregá un mini-resumen.
- Respuestas siempre en español (rioplatense).
- No cambies datos en la base (solo lectura). Para SQL, usá consultas SELECT.
- GRÁFICOS/IMÁGENES → usa "grafico_tickets_vendidos"
   Usuario dice: "gráfico", "grafico", "visualiza", "muestra gráficamente", "chart"
   → Ejecuta DIRECTAMENTE "grafico_tickets_vendidos" (no requiere parámetros)

Enrutamiento de herramientas (usá el **nombre exacto**):
- **disponibilidad_evento** → saber capacidad, vendidos y cupos disponibles de un evento (input: id o parte del nombre).
- **lista_eventos** → listar id, nombre, fecha y capacidad de eventos.
- **recomendar_por_presupuesto** → sugerir eventos/tipos de ticket con precio ≤ presupuesto (input: monto en pesos, ej. "15000").
- **politica_vigente** → ver política de reembolso activa.
- **sql_select** → ejecutar una SELECT libre (solo lectura) si el usuario pide una consulta específica.
- **resumen_ejecutivo** → dado un CSV de salida de 'sql_select', generar un resumen ejecutivo de hallazgos.
- **calc** → cálculos aritméticos simples.
- **organizador_mas_eventos** -> Para cuando se pregunte por los organizadores con más eventos.
- **organizadores_por_recaudacion** -> Utiliza esta tool cuando soliciten los organizadores con mayores cantidad de tickets vendidos
-**categoria_evento** -> Utiliza esta tool cuando soliciten eventos pertenecientes a una categoria

Buenas prácticas de salida:
- Cuando muestres resultados tabulares, presentá una tabla legible y un **mini-resumen** (2–4 bullets).
- Si la tool devuelve un CSV al final, **no lo pegues entero** salvo que el usuario lo pida; resumí y ofrecé compartirlo.
- Si no hay resultados, decílo explícitamente y sugerí un filtro alternativo.

Available tools:
{tools}

Question: {input}
{agent_scratchpad}
"""

prompt = PromptTemplate.from_template(template)


## Creación del Agente y del Ejecutor

En esta sección se instancia el **agente inteligente ReAct**, combinando el modelo de lenguaje (LLM), el conjunto de herramientas (`TOOLS`) y el prompt definido previamente.  

Primero se inicializa el modelo de lenguaje mediante la función `make_llm()`. Luego se crea el agente ReAct utilizando `create_react_agent()`, integrando el modelo, las herramientas y las reglas de comportamiento definidas en el prompt.  

Finalmente, el agente se encapsula dentro de un `AgentExecutor`, el cual se encarga de coordinar el ciclo de razonamiento del agente, manejar errores de formato y limitar la cantidad máxima de iteraciones a diez.  

In [11]:
llm = make_llm()  # gpt-4o-mini por default arriba
react_agent = create_react_agent(llm=llm, tools=TOOLS, prompt=prompt)
agent_executor = AgentExecutor(agent=react_agent, tools=TOOLS, verbose=True, handle_parsing_errors=True, max_iterations=10)


## Incorporación de memoria y función de interacción

En esta última parte se agrega una **memoria de conversación** al agente, permitiendo que mantenga el contexto entre distintas consultas realizadas por el mismo usuario dentro de una sesión.  

Para ello, se usa un diccionario llamado `store` que actúa como un contenedor de historiales (`ChatMessageHistory`), indexado por un identificador de sesión (`SESSION_ID`). La función `_get_history()` se encarga de crear o recuperar el historial correspondiente a cada sesión activa.  

Luego, el agente se envuelve en un `RunnableWithMessageHistory`, que combina el ejecutor del agente con la capacidad de recordar mensajes anteriores mediante las claves `input_messages_key` y `history_messages_key`.  

Por ultimo, se define la función `ask(msg)`, que permite interactuar con el agente enviándole un mensaje en lenguaje natural y mostrar en pantalla la respuesta generada.


In [14]:
store = {}  # {session_id: ChatMessageHistory()}

def _get_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

agent_with_memory = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: _get_history(session_id),
    input_messages_key="input",
    history_messages_key="chat_history",
)

SESSION_ID = "session_demo_ticketera"

def ask(msg: str):
    out = agent_with_memory.invoke(
        {"input": msg},
        config={"configurable": {"session_id": SESSION_ID}},
    )
    print("\n--- RESPUESTA ---\n", out["output"])
    return out


## Conclusiones finales

El sistema implementado demuestra cómo un agente basado en modelos de lenguaje puede interactuar con una base de datos real para resolver consultas complejas en lenguaje natural.  

La aplicación práctica se centra en la gestión de eventos y tickets, donde el agente puede listar eventos, calcular disponibilidad, analizar políticas vigentes, generar gráficos de ventas y ofrecer resúmenes automáticos.  

De este modo, se integra razonamiento, acción y comunicación natural en una misma arquitectura, mostrando el potencial de la IA aplicada a entornos de información reales.

Durante el desarrollo se tomaron decisiones orientadas a la seguridad, modularidad y claridad del sistema. Entre ellas, se optó por restringir las consultas SQL a solo lectura mediante la función `safe_select()`, utilizar LangChain como framework de orquestación del agente ReAct, e integrar un modelo de lenguaje OpenAI por su estabilidad y soporte.  

Los principales desafíos técnicos incluyeron la integración entre el modelo LLM y la base de datos Supabase, el manejo de errores en consultas SQL dinámicas generadas por el LLM y la correcta definición de las tools que utiliza el agente para responder a las peticiones del usuario.



## Ejemplos de uso

In [ ]:
ask("¿Hay cupos para el evento 1 ?")



> Entering new AgentExecutor chain...
Thought: Voy a verificar la disponibilidad del evento con ID 1 para saber cuántos tickets han sido vendidos y cuántos cupos están disponibles.  
Action: disponibilidad_evento  
Action Input: "1"  |   idEvento | nombre        |   capacidadMax |   tickets_emitidos |   cupos_disponibles |
|-----------:|:--------------|---------------:|-------------------:|--------------------:|
|          1 | Oasis Reunion |           5000 |                  3 |                4997 |Observation: La disponibilidad del evento "Oasis Reunion" con ID 1 es la siguiente:
- Capacidad máxima: 5000
- Tickets emitidos: 3
- Cupos disponibles: 4997

Final Answer: Sí, hay cupos disponibles para el evento "Oasis Reunion". Actualmente, quedan 4997 tickets disponibles.

> Finished chain.

--- RESPUESTA ---
 Sí, hay cupos disponibles para el evento "Oasis Reunion". Actualmente, quedan 4997 tickets disponibles.


{'input': '¿Hay cupos para el evento 1 ?',
 'chat_history': [],
 'output': 'Sí, hay cupos disponibles para el evento "Oasis Reunion". Actualmente, quedan 4997 tickets disponibles.'}

In [ ]:
ask("quiero que me des informacion y un resumen sobre los eventos que encuentres")



> Entering new AgentExecutor chain...
Thought: Para proporcionar información sobre los eventos, primero listaré todos los eventos disponibles en la base de datos. Luego, generaré un resumen ejecutivo con los hallazgos obtenidos.

Action: lista_eventos
Action Input: ''
[DEBUG] Ejecutando query segura:
 SELECT "idEvento","nombre","fechaHoraEvento","capacidadMax" FROM "Evento" ORDER BY "fechaHoraEvento" DESC
Vista previa (30 filas):
|   idEvento | nombre            | fechaHoraEvento           |   capacidadMax |
|-----------:|:------------------|:--------------------------|---------------:|
|          1 | Oasis Reunion     | 2025-12-15 21:00:00+00:00 |           5000 |
|          2 | Teatro Colón Gala | 2025-11-10 20:30:00+00:00 |           2000 |
|          5 | Bad Bunny         | 2025-10-13 22:21:08+00:00 |          20000 |
|          3 | River vs Boca     | 2025-10-05 17:00:00+00:00 |          60000 |
|          4 | JSConf Argentina  | 2025-09-30 09:00:00+00:00 |            800 |

CSV

{'input': 'quiero que me des informacion y un resumen sobre los eventos que encuentres',
 'chat_history': [HumanMessage(content='¿Hay cupos para el evento 1 ?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Sí, hay cupos disponibles para el evento "Oasis Reunion". Actualmente, quedan 4997 tickets disponibles.', additional_kwargs={}, response_metadata={})],
 'output': 'He recopilado información sobre los eventos programados para el año 2025. A continuación, se presenta un resumen ejecutivo con los detalles de cada evento:\n\n1. **Oasis Reunion**\n   - **Fecha y Hora:** 15 de diciembre de 2025, 21:00 (UTC)\n   - **Capacidad Máxima:** 5,000 personas\n\n2. **Teatro Colón Gala**\n   - **Fecha y Hora:** 10 de noviembre de 2025, 20:30 (UTC)\n   - **Capacidad Máxima:** 2,000 personas\n\n3. **Bad Bunny**\n   - **Fecha y Hora:** 13 de octubre de 2025, 22:21:08 (UTC)\n   - **Capacidad Máxima:** 20,000 personas\n\n4. **River vs Boca**\n   - **Fecha y Hora:** 5 de octubre de 202

In [15]:
ask("Mostrame un gráfico con la cantidad de tickets vendidos por evento.")



> Entering new AgentExecutor chain...
Action: grafico_tickets_vendidos
Action Input: [DEBUG] Ejecutando query para gráfico de tickets vendidos...
[DEBUG] Ejecutando query segura:
 SELECT e."nombre", COUNT(t."nroTicket") AS tickets_vendidos
        FROM "Evento" e
        LEFT JOIN "TipoTicket" tt ON e."idEvento" = tt."idEvento"
        LEFT JOIN "Ticket" t ON tt."idTipoTicket" = t."idTipoTicket"
        GROUP BY e."nombre"
        ORDER BY tickets_vendidos DESC
[DEBUG] Query completada. Filas: 5
Datos de tickets vendidos por evento:
           nombre  tickets_vendidos
    Oasis Reunion                 3
 JSConf Argentina                 2
    River vs Boca                 1
Teatro Colón Gala                 0
        Bad Bunny                 0

Gráfico guardado en: /tmp/grafico_tickets_evento.pngFinal Answer: He generado un gráfico que muestra la cantidad de tickets vendidos por evento. Los datos indican que el evento "Oasis Reunion" lidera con 3 tickets vendidos, seguido por "JSCon

{'input': 'Mostrame un gráfico con la cantidad de tickets vendidos por evento.',
 'chat_history': [],
 'output': 'He generado un gráfico que muestra la cantidad de tickets vendidos por evento. Los datos indican que el evento "Oasis Reunion" lidera con 3 tickets vendidos, seguido por "JSConf Argentina" con 2. Los eventos "River vs Boca", "Teatro Colón Gala" y "Bad Bunny" tienen 1, 0 y 0 tickets vendidos respectivamente. Si necesitas más información o el gráfico, házmelo saber.'}

In [ ]:
ask("¿Hay cupos para el evento 'River' ?")



> Entering new AgentExecutor chain...
Thought: Necesito verificar la disponibilidad de tickets para el evento "River vs Boca" para determinar cuántos cupos hay disponibles.  
Action: disponibilidad_evento  
Action Input: 'River vs Boca'  |   idEvento | nombre        |   capacidadMax |   tickets_emitidos |   cupos_disponibles |
|-----------:|:--------------|---------------:|-------------------:|--------------------:|
|          3 | River vs Boca |          60000 |                  1 |               59999 |Observation: La disponibilidad para el evento "River vs Boca" es la siguiente:
- **Capacidad Máxima:** 60,000 personas
- **Tickets Emitidos:** 1 ticket vendido
- **Cupos Disponibles:** 59,999 tickets

Final Answer: Sí, hay cupos disponibles para el evento "River vs Boca". Actualmente, quedan 59,999 tickets disponibles.

> Finished chain.

--- RESPUESTA ---
 Sí, hay cupos disponibles para el evento "River vs Boca". Actualmente, quedan 59,999 tickets disponibles.


{'input': "¿Hay cupos para el evento 'River' ?",
 'chat_history': [HumanMessage(content='¿Hay cupos para el evento 1 ?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Sí, hay cupos disponibles para el evento "Oasis Reunion". Actualmente, quedan 4997 tickets disponibles.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='quiero que me des informacion y un resumen sobre los eventos que encuentres', additional_kwargs={}, response_metadata={}),
  AIMessage(content='He recopilado información sobre los eventos programados para el año 2025. A continuación, se presenta un resumen ejecutivo con los detalles de cada evento:\n\n1. **Oasis Reunion**\n   - **Fecha y Hora:** 15 de diciembre de 2025, 21:00 (UTC)\n   - **Capacidad Máxima:** 5,000 personas\n\n2. **Teatro Colón Gala**\n   - **Fecha y Hora:** 10 de noviembre de 2025, 20:30 (UTC)\n   - **Capacidad Máxima:** 2,000 personas\n\n3. **Bad Bunny**\n   - **Fecha y Hora:** 13 de octubre de 2025, 22:21:08 (U

In [ ]:
ask("¿Hay cupo para el evento 'Oasis' ?")



> Entering new AgentExecutor chain...
Thought: Voy a verificar la disponibilidad del evento "Oasis Reunion" para determinar cuántos tickets quedan disponibles.  
Action: disponibilidad_evento  
Action Input: "Oasis"  |   idEvento | nombre        |   capacidadMax |   tickets_emitidos |   cupos_disponibles |
|-----------:|:--------------|---------------:|-------------------:|--------------------:|
|          1 | Oasis Reunion |           5000 |                  3 |                4997 |Observation: La disponibilidad del evento "Oasis Reunion" es la siguiente:

|   idEvento | nombre        |   capacidadMax |   tickets_emitidos |   cupos_disponibles |
|-----------:|:--------------|---------------:|-------------------:|--------------------:|
|          1 | Oasis Reunion |           5000 |                  3 |                4997 |

Final Answer: Sí, hay cupos disponibles para el evento "Oasis Reunion". Actualmente, quedan 4,997 tickets disponibles.

> Finished chain.

--- RESPUESTA ---
 S

{'input': "¿Hay cupo para el evento 'Oasis' ?",
 'chat_history': [HumanMessage(content='¿Hay cupos para el evento 1 ?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Sí, hay cupos disponibles para el evento "Oasis Reunion". Actualmente, quedan 4997 tickets disponibles.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='quiero que me des informacion y un resumen sobre los eventos que encuentres', additional_kwargs={}, response_metadata={}),
  AIMessage(content='He recopilado información sobre los eventos programados para el año 2025. A continuación, se presenta un resumen ejecutivo con los detalles de cada evento:\n\n1. **Oasis Reunion**\n   - **Fecha y Hora:** 15 de diciembre de 2025, 21:00 (UTC)\n   - **Capacidad Máxima:** 5,000 personas\n\n2. **Teatro Colón Gala**\n   - **Fecha y Hora:** 10 de noviembre de 2025, 20:30 (UTC)\n   - **Capacidad Máxima:** 2,000 personas\n\n3. **Bad Bunny**\n   - **Fecha y Hora:** 13 de octubre de 2025, 22:21:08 (UT

In [ ]:
ask("¿hay disponibilidad para el evento 1 ")



> Entering new AgentExecutor chain...
Thought: El usuario pregunta sobre la disponibilidad del evento 1. Para obtener la información precisa, utilizaré la herramienta de disponibilidad de eventos, buscando específicamente el evento "Oasis Reunion", que es el primero en la lista de eventos.  
Action: disponibilidad_evento  
Action Input: "Oasis Reunion"  |   idEvento | nombre        |   capacidadMax |   tickets_emitidos |   cupos_disponibles |
|-----------:|:--------------|---------------:|-------------------:|--------------------:|
|          1 | Oasis Reunion |           5000 |                  3 |                4997 |Final Answer: Sí, hay disponibilidad para el evento "Oasis Reunion". Actualmente, quedan 4,997 tickets disponibles de un total de 5,000. Se han vendido 3 tickets hasta el momento. Si necesitas más información, no dudes en preguntar.

> Finished chain.

--- RESPUESTA ---
 Sí, hay disponibilidad para el evento "Oasis Reunion". Actualmente, quedan 4,997 tickets disponibl

{'input': '¿hay disponibilidad para el evento 1 ',
 'chat_history': [HumanMessage(content='¿Hay cupos para el evento 1 ?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Sí, hay cupos disponibles para el evento "Oasis Reunion". Actualmente, quedan 4997 tickets disponibles.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='quiero que me des informacion y un resumen sobre los eventos que encuentres', additional_kwargs={}, response_metadata={}),
  AIMessage(content='He recopilado información sobre los eventos programados para el año 2025. A continuación, se presenta un resumen ejecutivo con los detalles de cada evento:\n\n1. **Oasis Reunion**\n   - **Fecha y Hora:** 15 de diciembre de 2025, 21:00 (UTC)\n   - **Capacidad Máxima:** 5,000 personas\n\n2. **Teatro Colón Gala**\n   - **Fecha y Hora:** 10 de noviembre de 2025, 20:30 (UTC)\n   - **Capacidad Máxima:** 2,000 personas\n\n3. **Bad Bunny**\n   - **Fecha y Hora:** 13 de octubre de 2025, 22:21:08 

In [ ]:
ask("¿hay cupos para el evento 1? ")



> Entering new AgentExecutor chain...
Thought: El usuario pregunta sobre la disponibilidad de cupos para el evento 1. Debo verificar la disponibilidad del evento "Oasis Reunion", que es el evento 1 en la lista proporcionada anteriormente. 

Action: disponibilidad_evento  
Action Input: "Oasis Reunion"  
|   idEvento | nombre        |   capacidadMax |   tickets_emitidos |   cupos_disponibles |
|-----------:|:--------------|---------------:|-------------------:|--------------------:|
|          1 | Oasis Reunion |           5000 |                  3 |                4997 |Final Answer: Sí, hay cupos disponibles para el evento "Oasis Reunion". Actualmente, quedan 4,997 tickets disponibles de un total de 5,000. Se han vendido 3 tickets hasta el momento. Si necesitas más información, no dudes en preguntar.

> Finished chain.

--- RESPUESTA ---
 Sí, hay cupos disponibles para el evento "Oasis Reunion". Actualmente, quedan 4,997 tickets disponibles de un total de 5,000. Se han vendido 3 tic

{'input': '¿hay cupos para el evento 1? ',
 'chat_history': [HumanMessage(content='¿Hay cupos para el evento 1 ?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Sí, hay cupos disponibles para el evento "Oasis Reunion". Actualmente, quedan 4997 tickets disponibles.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='quiero que me des informacion y un resumen sobre los eventos que encuentres', additional_kwargs={}, response_metadata={}),
  AIMessage(content='He recopilado información sobre los eventos programados para el año 2025. A continuación, se presenta un resumen ejecutivo con los detalles de cada evento:\n\n1. **Oasis Reunion**\n   - **Fecha y Hora:** 15 de diciembre de 2025, 21:00 (UTC)\n   - **Capacidad Máxima:** 5,000 personas\n\n2. **Teatro Colón Gala**\n   - **Fecha y Hora:** 10 de noviembre de 2025, 20:30 (UTC)\n   - **Capacidad Máxima:** 2,000 personas\n\n3. **Bad Bunny**\n   - **Fecha y Hora:** 13 de octubre de 2025, 22:21:08 (UTC)\n 

In [ ]:
ask("¿Cuál es la política de reembolso vigente hoy?")




> Entering new AgentExecutor chain...
Thought: Necesito consultar la política de reembolso más reciente para proporcionar la información solicitada.  
Action: politica_vigente  
Action Input: ''  [DEBUG] Ejecutando query segura:
 SELECT "fechaVigencia","tiempoReembolso" FROM "Politica" ORDER BY "fechaVigencia" DESC LIMIT 1
Política vigente desde 2025-02-01: reembolso dentro de 3 días.Final Answer: La política de reembolso vigente desde el 1 de febrero de 2025 establece que se puede solicitar un reembolso dentro de los 3 días posteriores a la compra. Si necesitas más información, no dudes en preguntar.

> Finished chain.

--- RESPUESTA ---
 La política de reembolso vigente desde el 1 de febrero de 2025 establece que se puede solicitar un reembolso dentro de los 3 días posteriores a la compra. Si necesitas más información, no dudes en preguntar.


{'input': '¿Cuál es la política de reembolso vigente hoy?',
 'chat_history': [HumanMessage(content='¿Hay cupos para el evento 1 ?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Sí, hay cupos disponibles para el evento "Oasis Reunion". Actualmente, quedan 4997 tickets disponibles.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='quiero que me des informacion y un resumen sobre los eventos que encuentres', additional_kwargs={}, response_metadata={}),
  AIMessage(content='He recopilado información sobre los eventos programados para el año 2025. A continuación, se presenta un resumen ejecutivo con los detalles de cada evento:\n\n1. **Oasis Reunion**\n   - **Fecha y Hora:** 15 de diciembre de 2025, 21:00 (UTC)\n   - **Capacidad Máxima:** 5,000 personas\n\n2. **Teatro Colón Gala**\n   - **Fecha y Hora:** 10 de noviembre de 2025, 20:30 (UTC)\n   - **Capacidad Máxima:** 2,000 personas\n\n3. **Bad Bunny**\n   - **Fecha y Hora:** 13 de octubre de 2025, 

In [ ]:
ask("Tengo 7000 pesos, ¿qué evento y tipo de ticket podría comprar? Devolveme una tabla.")



> Entering new AgentExecutor chain...
Thought: Voy a utilizar la herramienta "recomendar_por_presupuesto" para identificar eventos y tipos de tickets que se ajusten al presupuesto de 7000 pesos. Esto me permitirá presentar una tabla con las opciones disponibles.

Action: recomendar_por_presupuesto  
Action Input: '7000'
[DEBUG] Ejecutando query segura:
 SELECT e."idEvento", e."nombre" AS evento, e."fechaHoraEvento",
           tt."idTipoTicket", tt."tipo", tt."precio"
    FROM "TipoTicket" tt
    JOIN "Evento" e ON e."idEvento" = tt."idEvento"
    WHERE tt."precio" <= 7000.0
    ORDER BY e."fechaHoraEvento" ASC, tt."precio" DESC
Vista previa (30 filas):
|   idEvento | evento           | fechaHoraEvento           |   idTipoTicket | tipo     |   precio |
|-----------:|:-----------------|:--------------------------|---------------:|:---------|---------:|
|          4 | JSConf Argentina | 2025-09-30 09:00:00+00:00 |              7 | Standard |     5000 |

CSV completo:
idEvento,evento,fe

{'input': 'Tengo 7000 pesos, ¿qué evento y tipo de ticket podría comprar? Devolveme una tabla.',
 'chat_history': [HumanMessage(content='¿Hay cupos para el evento 1 ?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Sí, hay cupos disponibles para el evento "Oasis Reunion". Actualmente, quedan 4997 tickets disponibles.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='quiero que me des informacion y un resumen sobre los eventos que encuentres', additional_kwargs={}, response_metadata={}),
  AIMessage(content='He recopilado información sobre los eventos programados para el año 2025. A continuación, se presenta un resumen ejecutivo con los detalles de cada evento:\n\n1. **Oasis Reunion**\n   - **Fecha y Hora:** 15 de diciembre de 2025, 21:00 (UTC)\n   - **Capacidad Máxima:** 5,000 personas\n\n2. **Teatro Colón Gala**\n   - **Fecha y Hora:** 10 de noviembre de 2025, 20:30 (UTC)\n   - **Capacidad Máxima:** 2,000 personas\n\n3. **Bad Bunny**\n   - **Fe

In [ ]:
ask("Quiero ir al evento de river vs boca, hay entradas disponibles?")



> Entering new AgentExecutor chain...
Thought: Verificaré la disponibilidad de entradas para el evento "River vs Boca" para proporcionarte la información necesaria.  
Action: disponibilidad_evento  
Action Input: "River vs Boca"  |   idEvento | nombre        |   capacidadMax |   tickets_emitidos |   cupos_disponibles |
|-----------:|:--------------|---------------:|-------------------:|--------------------:|
|          3 | River vs Boca |          60000 |                  1 |               59999 |Observation: La disponibilidad para el evento "River vs Boca" es la siguiente:

|   idEvento | nombre        |   capacidadMax |   tickets_emitidos |   cupos_disponibles |
|-----------:|:--------------|---------------:|-------------------:|--------------------:|
|          3 | River vs Boca |          60000 |                  1 |               59999 |

- **Capacidad máxima:** 60,000 personas.
- **Tickets emitidos:** 1 ticket vendido.
- **Cupos disponibles:** 59,999 tickets.

Final Answer: Sí,

{'input': 'Quiero ir al evento de river vs boca, hay entradas disponibles?',
 'chat_history': [HumanMessage(content='¿Hay cupos para el evento 1 ?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Sí, hay cupos disponibles para el evento "Oasis Reunion". Actualmente, quedan 4997 tickets disponibles.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='quiero que me des informacion y un resumen sobre los eventos que encuentres', additional_kwargs={}, response_metadata={}),
  AIMessage(content='He recopilado información sobre los eventos programados para el año 2025. A continuación, se presenta un resumen ejecutivo con los detalles de cada evento:\n\n1. **Oasis Reunion**\n   - **Fecha y Hora:** 15 de diciembre de 2025, 21:00 (UTC)\n   - **Capacidad Máxima:** 5,000 personas\n\n2. **Teatro Colón Gala**\n   - **Fecha y Hora:** 10 de noviembre de 2025, 20:30 (UTC)\n   - **Capacidad Máxima:** 2,000 personas\n\n3. **Bad Bunny**\n   - **Fecha y Hora:** 13 de 

In [ ]:
ask("Cuales son los organizadores con más eventos?")



> Entering new AgentExecutor chain...
Thought: Necesito obtener información sobre los organizadores que tienen la mayor cantidad de eventos registrados. Para ello, utilizaré la herramienta correspondiente para obtener un ranking de organizadores con más eventos.

Action: organizador_mas_eventos
Action Input: '' [DEBUG] Ejecutando query segura:
 SELECT o."nombre" AS organizador, COUNT(e."idEvento") AS cantidad_eventos FROM "Evento" e LEFT JOIN "Organizacion" o ON e."idOrganizacion" = o."idOrganizacion" GROUP BY o."nombre" ORDER BY cantidad_eventos DESC
[DEBUG] Ejecutando query segura:
 SELECT o."nombre" AS organizador, COUNT(e."idEvento") AS cantidad_eventos FROM "Evento" e LEFT JOIN "Organizacion" o ON e."idOrganizacion" = o."idOrganizacion" GROUP BY o."nombre" ORDER BY cantidad_eventos DESC
Vista previa (30 filas):
| organizador        |   cantidad_eventos |
|:-------------------|-------------------:|
| Eventos Rosario SA |                  3 |
| Shows BA SRL       |                

{'input': 'Cuales son los organizadores con más eventos?',
 'chat_history': [HumanMessage(content='¿Hay cupos para el evento 1 ?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Sí, hay cupos disponibles para el evento "Oasis Reunion". Actualmente, quedan 4997 tickets disponibles.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='quiero que me des informacion y un resumen sobre los eventos que encuentres', additional_kwargs={}, response_metadata={}),
  AIMessage(content='He recopilado información sobre los eventos programados para el año 2025. A continuación, se presenta un resumen ejecutivo con los detalles de cada evento:\n\n1. **Oasis Reunion**\n   - **Fecha y Hora:** 15 de diciembre de 2025, 21:00 (UTC)\n   - **Capacidad Máxima:** 5,000 personas\n\n2. **Teatro Colón Gala**\n   - **Fecha y Hora:** 10 de noviembre de 2025, 20:30 (UTC)\n   - **Capacidad Máxima:** 2,000 personas\n\n3. **Bad Bunny**\n   - **Fecha y Hora:** 13 de octubre de 2025, 2

In [ ]:
ask("¿Qué organizador vendió más tickets?")



> Entering new AgentExecutor chain...
Thought: Para responder a la pregunta sobre qué organizador vendió más tickets, utilizaré la herramienta que me permite obtener información sobre los organizadores con mayor cantidad de tickets vendidos y el total recaudado. 

Action: organizadores_por_recaudacion

Action Input: '' 
[DEBUG] Ejecutando query segura:
 SELECT
          o."nombre" AS organizador,
          COUNT(t."nroTicket") AS tickets_vendidos,
          SUM(tt."precio") AS total_recaudado
        FROM "Organizacion" o
        JOIN "Evento" e ON o."idOrganizacion" = e."idOrganizacion"
        JOIN "TipoTicket" tt ON e."idEvento" = tt."idEvento"
        JOIN "Ticket" t ON tt."idTipoTicket" = t."idTipoTicket"
        GROUP BY o."nombre"
        ORDER BY total_recaudado DESC
Vista previa (30 filas):
| organizador        |   tickets_vendidos |   total_recaudado |
|:-------------------|-------------------:|------------------:|
| Eventos Rosario SA |                  4 |             700

{'input': '¿Qué organizador vendió más tickets?',
 'chat_history': [HumanMessage(content='¿Hay cupos para el evento 1 ?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Sí, hay cupos disponibles para el evento "Oasis Reunion". Actualmente, quedan 4997 tickets disponibles.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='quiero que me des informacion y un resumen sobre los eventos que encuentres', additional_kwargs={}, response_metadata={}),
  AIMessage(content='He recopilado información sobre los eventos programados para el año 2025. A continuación, se presenta un resumen ejecutivo con los detalles de cada evento:\n\n1. **Oasis Reunion**\n   - **Fecha y Hora:** 15 de diciembre de 2025, 21:00 (UTC)\n   - **Capacidad Máxima:** 5,000 personas\n\n2. **Teatro Colón Gala**\n   - **Fecha y Hora:** 10 de noviembre de 2025, 20:30 (UTC)\n   - **Capacidad Máxima:** 2,000 personas\n\n3. **Bad Bunny**\n   - **Fecha y Hora:** 13 de octubre de 2025, 22:21:08 (

In [ ]:
ask('Que eventos hay para la categoria deporte?')



> Entering new AgentExecutor chain...
Thought: Necesito buscar los eventos que pertenecen a la categoría "deporte" para proporcionar la información solicitada. 
Action: categoria_evento
Action Input: "deporte"[DEBUG] Ejecutando query segura:
 SELECT
            c."nombreCategoria" AS categoria,
            e."nombre" AS evento,
            e."descripcion"
        FROM "Evento" e
        INNER JOIN "Categoria" c ON e."idCategoria" = c."idCategoria"
        WHERE c."nombreCategoria" ILIKE '%deporte%'
        ORDER BY e."fechaHoraEvento" ASC
| categoria   | evento        | descripcion            |
|:------------|:--------------|:-----------------------|
| Deportes    | River vs Boca | Superclásico argentino |Action: categoria_evento  
Action Input: "deporte"  [DEBUG] Ejecutando query segura:
 SELECT
            c."nombreCategoria" AS categoria,
            e."nombre" AS evento,
            e."descripcion"
        FROM "Evento" e
        INNER JOIN "Categoria" c ON e."idCategoria" = c."i

{'input': 'Que eventos hay para la categoria deporte?',
 'chat_history': [HumanMessage(content='¿Hay cupos para el evento 1 ?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Sí, hay cupos disponibles para el evento "Oasis Reunion". Actualmente, quedan 4997 tickets disponibles.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='quiero que me des informacion y un resumen sobre los eventos que encuentres', additional_kwargs={}, response_metadata={}),
  AIMessage(content='He recopilado información sobre los eventos programados para el año 2025. A continuación, se presenta un resumen ejecutivo con los detalles de cada evento:\n\n1. **Oasis Reunion**\n   - **Fecha y Hora:** 15 de diciembre de 2025, 21:00 (UTC)\n   - **Capacidad Máxima:** 5,000 personas\n\n2. **Teatro Colón Gala**\n   - **Fecha y Hora:** 10 de noviembre de 2025, 20:30 (UTC)\n   - **Capacidad Máxima:** 2,000 personas\n\n3. **Bad Bunny**\n   - **Fecha y Hora:** 13 de octubre de 2025, 22:2

In [ ]:
ask("Que eventos hay para la categoria concierto")



> Entering new AgentExecutor chain...
Thought: El usuario ya ha consultado sobre eventos en la categoría "concierto" y ha recibido información sobre un evento específico. Sin embargo, para asegurarme de que no haya más eventos disponibles en esta categoría, procederé a listar todos los eventos relacionados con "concierto".

Action: categoria_evento  
Action Input: "concierto"  [DEBUG] Ejecutando query segura:
 SELECT
            c."nombreCategoria" AS categoria,
            e."nombre" AS evento,
            e."descripcion"
        FROM "Evento" e
        INNER JOIN "Categoria" c ON e."idCategoria" = c."idCategoria"
        WHERE c."nombreCategoria" ILIKE '%concierto%'
        ORDER BY e."fechaHoraEvento" ASC
| categoria   | evento        | descripcion                  |
|:------------|:--------------|:-----------------------------|
| Conciertos  | Bad Bunny     | recital bad bunny            |
| Conciertos  | Oasis Reunion | Concierto histórico de Oasis |Observation: He encontrado do

{'input': 'Que eventos hay para la categoria concierto',
 'chat_history': [HumanMessage(content='Que eventos hay para la categoria deporte?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='El único evento disponible en la categoría "deporte" es "River vs Boca", que es el Superclásico argentino.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Que eventos hay para la categoria concierto', additional_kwargs={}, response_metadata={}),
  AIMessage(content='El evento disponible en la categoría "concierto" es "Oasis Reunion", un concierto histórico de Oasis.', additional_kwargs={}, response_metadata={})],
 'output': 'Los eventos disponibles en la categoría "concierto" son: "Bad Bunny", un recital, y "Oasis Reunion", un concierto histórico de Oasis.'}